In [2]:
cd /private/home/dpf/projects/codex

/private/home/dpf/projects/codex


In [3]:
import sys

In [4]:
sys.path

['/private/home/dpf/projects/codex/notebooks',
 '/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python38.zip',
 '/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8',
 '/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/lib-dynload',
 '',
 '/private/home/dpf/.local/lib/python3.8/site-packages',
 '/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages',
 '/private/home/dpf/projects/Megatron-LM',
 '/private/home/dpf/projects/htlm_2/fairseq-py',
 '/private/home/dpf/projects/tokenizers/bindings/python/py_src',
 '/private/home/dpf/projects/human-eval',
 '/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions',
 '/private/home/dpf/.ipython']

In [5]:
from models import *

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from he import make_parser, HUMAN_EVAL_STOP_WORDS
import shlex
import pprint

In [8]:
# BART

In [8]:
# from fairseq.models.bart import BARTModel

In [9]:
# bart_model = BARTModel.from_pretrained(
#     "/checkpoint/dpf/2022-03-10/ours.bart.pytorch-ddp-bs2.old-init.bart_1B.tps2048.uf4.mu49591.ms2.dr0.1.atdr0.1.actdr0.0.wd0.01.adam.beta9999.eps1e-08.clip1.0.lr0.0002.warm1500.fp16.poi_lam3.5.mask0.3.mask_lenspan-poisson.rpl_len1.rotate0.mask_rand0.1.ins0.perm_sen0.0.ngpu64",
#     "checkpoint_1_9000.pt",
#     "/checkpoint/dpf/2022-03-10/ours.bart.pytorch-ddp-bs2.old-init.bart_1B.tps2048.uf4.mu49591.ms2.dr0.1.atdr0.1.actdr0.0.wd0.01.adam.beta9999.eps1e-08.clip1.0.lr0.0002.warm1500.fp16.poi_lam3.5.mask0.3.mask_lenspan-poisson.rpl_len1.rotate0.mask_rand0.1.ins0.perm_sen0.0.ngpu64",
#     bpe="gpt2_pretokenization_newlines_only",
# )

In [10]:
# encoded = bart_model.encode("def count_words(filename):\n")

In [11]:
# CM

In [9]:
parser = make_parser()

In [10]:
args = parser.parse_args(shlex.split("--model_name /checkpoint/dpf/models/cm-6B-armen/checkpoint_last_consolidated.pt --num_candidates_generated 1 --num_candidates_evaluated 1 --batch_size 3 --temperature 0.2 --top_p 0.95 --output_filename expts/he/lm-1.3B-ourtok_last_ncg-1_temp-0.2/prompt_pyfile_normalized/samples.jsonl --response_filename expts/he/lm-1.3B-ourtok_last_ncg-1_temp-0.2/prompt_pyfile_normalized/responses.pkl --prompt_prefix '<| file ext=.py |>' --tokenizer_name gpt2_pretokenization_newlines_only"))

In [11]:
pprint.pprint(vars(args))

{'batch_size': 3,
 'beam': 1,
 'cached_responses': False,
 'candidate_scoring': 'mean',
 'git_status': False,
 'max_tokens': 450,
 'model_name': '/checkpoint/dpf/models/cm-6B-armen/checkpoint_last_consolidated.pt',
 'multiple_cached_responses_filenames': None,
 'num_candidates_evaluated': 1,
 'num_candidates_generated': 1,
 'num_problems': None,
 'output_filename': 'expts/he/lm-1.3B-ourtok_last_ncg-1_temp-0.2/prompt_pyfile_normalized/samples.jsonl',
 'prompt_prefix': '<| file ext=.py |>',
 'remove_test_cases': False,
 'response_filename': 'expts/he/lm-1.3B-ourtok_last_ncg-1_temp-0.2/prompt_pyfile_normalized/responses.pkl',
 'temperature': 0.2,
 'tokenizer_name': 'gpt2_pretokenization_newlines_only',
 'top_p': 0.95,
 'verbose': False}


In [12]:
model = make_model(args)

model_root_dir: /checkpoint/dpf/models/cm-6B-armen
model_basename: checkpoint_last_consolidated.pt
dictionary size: 50518
Detected CUDA files, patching ldflags
Emitting ninja build file /private/home/dpf/projects/Megatron-LM/megatron/fused_kernels/build/build.ninja...
Building extension module fused_mix_prec_layer_norm_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module fused_mix_prec_layer_norm_cuda...


In [20]:
[model.tokenizer.id_to_token(id-4) for id in model._encode("<sentinel:2>import re", strip_eos=True).tolist()]

['<sentinel:2>', 'importĠre']

In [13]:
import sys
sys.path[0] = '/private/home/dpf/projects/codex'

In [22]:
from human_eval.data import read_problems

In [23]:
problems = list(sorted(read_problems().items()))

In [24]:
task_id, problem = problems[0]
prompt = problem['prompt']

In [25]:
prompt

'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n'

In [26]:
args.num_candidates_generated

1

In [27]:
pdb on

Automatic pdb calling has been turned ON


In [28]:
model = make_model(args, cached_model=model.lm_model)

model_root_dir: /checkpoint/dpf/models/cm-6B-armen
model_basename: checkpoint_last_consolidated.pt


In [29]:
model.complete(prompt, stop_words=["\ndef"], sampling=True, n=5)

/private/home/dpf/projects/codex/models.py:620: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token_ids = torch.tensor(tokens)


{'choices': [{'text': '    smallest_value = numbers[0]\n    second_smallest_value = numbers[1]\n    for number in numbers:\n        if abs(number - smallest_value) < abs(number - second_smallest_value):\n            second_smallest_value = smallest_value\n            smallest_value = number\n        else:\n            second_smallest_value = number\n    return abs(smallest_value - threshold) < abs(second_smallest_value - threshold)\n\n',
   'logprobs': {'token_logprobs': [0.6997772455215454,
     0.016930216923356056,
     0.703140139579773,
     0.04975461587309837,
     0.3273661434650421,
     0.9991055727005005,
     0.9880794286727905,
     0.9564300179481506,
     0.6541919708251953,
     0.9918103218078613,
     0.9797477722167969,
     0.9973158836364746,
     0.9965722560882568,
     0.6786525845527649,
     0.7235873937606812,
     0.7021003365516663,
     0.5949846506118774,
     0.9999898672103882,
     0.999866783618927,
     0.937919557094574,
     0.9999516010284424,
   

In [26]:
lengths = torch.zeros(4).long()

In [34]:
lengths += zeros.long()

In [35]:
lengths

tensor([2, 2, 2, 2])

In [30]:
zeros = torch.zeros(4).bool()

In [33]:
zeros |= True

In [25]:
(~zeros).long()

tensor([0, 0, 0, 0])

In [93]:
from priming_generator import TopPSampling, GreedyDecoding

In [177]:
decoder = TopPSampling(model.lm_model, min_len=10, max_len=128, sampling_topp=0.95, temperature=0.6, show_tqdm=True).cuda()

In [158]:
greedy = GreedyDecoding(model.lm_model, min_len=10, max_len=128, temperature=1.0, show_tqdm=True).cuda()

In [159]:
encoded_prompt = model._encode('if __name__ == "__main__":').to(model.lm_model.device)

In [160]:
mct.size()

torch.Size([1, 43])

In [161]:
mcl

tensor([43], device='cuda:0')

In [162]:
encoded_stop_words = [model._encode(sw, strip_eos=True).tolist() for sw in model._extra_stop_words]

In [166]:
mct, mctlp, mcl = greedy.decode_multiple_candidates(
    # remove eos
    encoded_prompt[:-1],
    num_candidates=1,
    encoded_stop_words = [[decoder.eos]] + encoded_stop_words,
)

 24%|██████████████████████████████████████████████████▌                                                                                                                                                                   | 30/127 [00:02<00:07, 13.74it/s]


In [178]:
mct, mctlp, mcl = decoder.decode_multiple_candidates(
    # remove eos
    encoded_prompt[:-1],
    num_candidates=5,
    encoded_stop_words = [[decoder.eos]] + encoded_stop_words,
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 127/127 [00:19<00:00,  6.50it/s]


In [183]:
model.batch_size = 5

In [184]:
response = model.complete(prompt, HUMAN_EVAL_STOP_WORDS, sampling=True, max_tokens=200, n=5)

In [186]:
for ix, c in enumerate(response['choices']):
    print(ix)
    print(c['text'])
    print()

0
    length = len(numbers)
    for i in range(length - 1):
        if abs(numbers[i] - numbers[i + 1]) < threshold:
            return True
    return False



1
    length = len(numbers)
    if length < 2:
        return False

    numbers.sort()
    for i in range(length-1):
        if abs(numbers[i] - numbers[i+1]) > threshold:
            return True
    return False



2
    length = len(numbers)
    if length < 2:
        return False

    # sort list
    numbers.sort()

    # check if first two numbers are closer to each other than
    # the threshold
    if abs(numbers[0] - numbers[1]) < threshold:
        return True

    # check if next two numbers are closer to each other than
    # the threshold
    if abs(numbers[1] - numbers[2]) < threshold:
        return True

    # check if next two numbers are closer to each other than
    # the threshold
    if abs(numbers[2] - numbers[3]) < threshold:
        return True

    # check if next two numbers are closer to each other tha

In [49]:
completions, response = model.rank_completions(
    prompt, HUMAN_EVAL_STOP_WORDS,
    max_tokens=200,
    n=5,
    cached_response=None,
    scoring='mean',
    sampling=True,
    temperature=0.6,
    top_p=0.95,
    beam=1,
)

In [51]:
model.rank_infills?

In [54]:
model.prompt_prefix = None

In [56]:
model.rank_infills?

In [61]:
import models

In [63]:
pdb on

Automatic pdb calling has been turned ON


In [86]:
model = make_model(args, cached_model=model.lm_model)
model.prompt_prefix = None

model_root_dir: /checkpoint/dpf/models/cm-6B-armen
model_basename: checkpoint_last_consolidated.pt


In [111]:
prefix = 'def count_words(filename):\n    """'
suffix = '    """\n    with open(filename) as f:\n        return Counter(f.read().split())'

In [88]:
print(prefix)

def count_words(filename):
   """


In [89]:
print(suffix)

    """
    with open(filename) as f:
        return Counter(f.read().split())


In [97]:
truncation_parameters = [TruncationParameters(1, suffix)]

In [102]:
r['choices'][0].keys()

dict_keys(['complete', 'infills_untruncated', 'ids', 'logprobs'])

In [106]:
print(truncate_num_lines(r['choices'][0]['infills_untruncated'][0], max_num_lines=1))

   Count the number of words in a file


In [112]:
r = model.infill([prefix, suffix],
             truncation_parameters=truncation_parameters,
             verbose=True,
             sampling=True,
             temperature=0.0,
             top_p=0.95,
             n=1,
             max_tokens=40,
             beam=1
            )

def count_words(filename):
    """<sentinel:0>

In [113]:
print(''.join(r['choices'][0]['complete']))

def count_words(filename):
    """
    Count the number of words in a file.
    """
    with open(filename) as f:
        return len(f.read().split())


def count_characters(filename):
    """
    Count the number of characters in a file.
    """
    """
    with open(filename) as f:
        return Counter(f.read().split())


In [114]:
completions, response = model.rank_infills(
    [prefix, suffix], 
    truncation_parameters=truncation_parameters,
    verbose=True,
    bidirectional_scoring=False,
    bidirectional_generation=True,
    max_tokens=40,
    n=1,
    cached_response=None,
    scoring='random',
    sampling=True,
    temperature=0.0,
    top_p=0.95,
    beam=1,
)

def count_words(filename):
    """<sentinel:0>--prefix:--
def count_words(filename):
    """
--infill (truncated):--
    Count the number of words in a file.
--infill (untruncated):--

    Count the number of words in a file.
    """
    with open(filename) as f:
        return len(f.read().split())


def count_characters(filename):
    """
    Count the number of characters in a file.
    """

--suffix:--
    """
    with open(filename) as f:
        return Counter(f.read().split())


In [115]:
print(completions[0]['complete'])

def count_words(filename):
    """
    Count the number of words in a file.
    """
    with open(filename) as f:
        return Counter(f.read().split())


In [108]:
print(completions[0]['text'])

KeyError: 'text'

> /tmp/ipykernel_1419960/3152976465.py(1)<module>()
----> 1 print(completions[0]['text'])

ipdb> q


In [42]:
print(completions[0]['text'])

    length = len(numbers)
    if length < 2:
        return False
    i = 0
    while i < length - 1:
        if abs(numbers[i] - numbers[i + 1]) < threshold:
            return True
        i += 1
    return False




In [39]:
ids = torch.tensor([    4,  7003,   914,    35,  9444,     6,   205, 48158,   536,   380,
         4732, 14180,  1375,   484,  1726, 14641, 37417,  6342,  8660, 18567,
          205,  7749, 13161,   366,  2649,  2433,  1726,   463,   937,  2193,
         3510, 34254,  1304,   374,  1222,  2065,  2370, 20711,   205,   264,
         2649, 13349,   696,   205,  2296,  4732, 14180,  1375,  8877,  6280,
        22672,  1201, 13102, 12444,   205,   264,   690,   205,  2296,  4732,
        14180,  1375,  8877,  6280,   845,  3554, 41762,  2409,   546, 41091,
          845,   589,  3024,  2644,   205, 28660,   205,   768,   205])

In [40]:
print(model.lm_model.decode(ids))

<| file ext=.py |>
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



In [34]:
model._decode(ids)

'<| file ext=.py |>\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n'

In [30]:
completions

[{'text': '', 'logprobs': {'token_logprobs': [], 'tokens': []}}]

In [25]:
model.lm_model.task.source_dictionary.symbols?

In [47]:
completions

[{'text': '', 'logprobs': {'token_logprobs': [], 'tokens': []}}]

In [43]:
# completions, response = model.rank_completions(
#     prompt, HUMAN_EVAL_STOP_WORDS,
#     max_tokens=450,
#     n=args.num_candidates_generated,
#     cached_response=None,
#     scoring=args.candidate_scoring,
#     temperature=args.temperature,
#     top_p=args.top_p,
# )

In [41]:
completions

[{'text': '', 'logprobs': {'token_logprobs': [], 'tokens': []}}]

In [14]:
problem

{'task_id': 'HumanEval/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'entry_point': 'has_close_elements',
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9,

In [15]:
from fairseq.priming_generator import TopPSampling, TopKSampling

In [70]:
top_p = TopPSampling(model.lm_model, 128, 256, 0.95, 1.0)

In [46]:
top_k = TopPSampling(model.lm_model, 128, 256, 0.95, 0.2)

In [5]:
### INFILLS

In [30]:
import infill_evaluation

In [31]:
from he import generate_he_infill_problems

In [32]:
from models import TruncationParameters

In [33]:
infill_parser = infill_evaluation.make_parser()

In [34]:
infill_args = infill_parser.parse_args(shlex.split("--model_name /checkpoint/dpf/models/cm-6B-armen/cm-6B-ourtok/best.pt \
  --tokenizer_name gpt2_pretokenization_newlines_only  \
  --candidate_scoring random \
  --batch_size 10 \
  --eval_type one_line \
  --truncation_heuristics num_lines suffix \
  --temperature 0.2 \
  --num_candidates 1 \
  --bidirectional_generation \
  "))

In [35]:
pprint.pprint(vars(infill_args))

{'batch_size': 10,
 'beam': 1,
 'bidirectional_generation': True,
 'bidirectional_scoring': False,
 'candidate_scoring': 'random',
 'eval_type': 'one_line',
 'evaluate_only': False,
 'git_status': False,
 'max_infill_attempts': 1,
 'max_tokens': 450,
 'model_name': '/checkpoint/dpf/models/cm-6B-armen/cm-6B-ourtok/best.pt',
 'num_candidates': 1,
 'prompt_prefix': None,
 'result_base_path': None,
 'temperature': 0.2,
 'tokenizer_name': 'gpt2_pretokenization_newlines_only',
 'top_p': 0.95,
 'truncation_heuristics': ['num_lines', 'suffix']}


In [38]:
make_model?

In [39]:
# no caching
# model = make_model(infill_args)
# with caching
model = make_model(infill_args, cached_model=model.lm_model)

model_root_dir: /checkpoint/dpf/models/cm-6B-armen/cm-6B-ourtok
model_basename: best.pt


In [41]:
model

In [42]:
from human_eval.data import read_problems

In [43]:
humaneval_problems = read_problems()

In [44]:
infill_problems = generate_he_infill_problems(infill_args, eval_type="one_line")

In [45]:
infill_problems = list(infill_problems)

In [48]:
model.infill?

In [ ]:
model.infill()

In [49]:
key, probs = infill_problems[0]
problem = probs[3]

In [50]:
problem

{'task_id': 'HumanEval/0',
 'num_before': 3,
 'num_after': 4,
 'missing_lines': '                distance = abs(elem - elem2)',
 'prompt_parts': ['from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n',
  '                if distance < threshold:\n                    return True\n\n    return False'],
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    re

In [15]:
model._encode("def count_words(filename):\n")[:-1] - 4

tensor([  376,  5003, 23193, 18844,   201])

In [51]:
prompt_parts = problem["prompt_parts"]
prefix, suffix = prompt_parts

In [71]:
model._decode([768])

'    """'

In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
response_stop = model.infill(prompt_parts, stop_words=['abs('], max_tokens=40)

In [61]:
[model._decode([x]) for x in response['choices'][0]['ids']]

['from typing import List',
 '\n\n\n',
 'def ',
 'has_',
 'close_',
 'element',
 's(',
 'number',
 's: List[',
 'float], ',
 'threshold',
 ': float',
 ') -> bool:',
 '\n',
 '    """ ',
 'Check if ',
 'in ',
 'given ',
 'list of ',
 'number',
 's, ',
 'are ',
 'any ',
 'two ',
 'numbers ',
 'clo',
 'ser',
 ' to ',
 'each ',
 'other ',
 'than',
 '\n',
 '    ',
 'given ',
 'threshol',
 'd.',
 '\n',
 '    >>> ',
 'has_',
 'close_',
 'element',
 's([',
 '1.0, ',
 '2.0, ',
 '3.',
 '0], ',
 '0.5)',
 '\n',
 '    ',
 'False',
 '\n',
 '    >>> ',
 'has_',
 'close_',
 'element',
 's([',
 '1.0, ',
 '2.',
 '8, ',
 '3.0, ',
 '4.',
 '0, ',
 '5.0, ',
 '2.',
 '0]',
 ', 0.',
 '3)',
 '\n',
 '    True',
 '\n',
 '    """',
 '\n',
 '    for ',
 'idx, ',
 'ele',
 'm in ',
 'enumerate(',
 'number',
 's):',
 '\n',
 '        for ',
 'idx',
 '2, ',
 'ele',
 'm2',
 ' in enumerate(',
 'number',
 's):',
 '\n',
 '            if ',
 'idx ',
 '!= ',
 'idx',
 '2:',
 '\n',
 '<sentinel:0>',
 '                if ',
 'dist

In [66]:
response['choices'][0]['complete']

['from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n',
 '                distance = abs(elem - elem2)\n',
 '                if distance < threshold:\n                    return True\n\n    return False']

In [65]:
response_stop['choices'][0]['complete']

['from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n',
 '                distance = ',
 '                if distance < threshold:\n                    return True\n\n    return False']

In [26]:
import numpy as np
import tqdm

In [29]:
from infill_evaluation import eval_result

In [31]:
# sanity check that we get 100% pass when feeding the original problem

# truncation_heuristics = ["num_lines", "suffix"]
truncation_heuristics = []
functional_results = []
infill_results = []
pbar = tqdm.tqdm(infill_problems)
for task_id, task_id_problems in pbar:
    if functional_results:
        avg_pass = np.mean([x["passed"] for x in functional_results])
        avg_exact = np.mean([x["exact_match"] for x in functional_results])
        pbar.set_postfix({'pass': avg_pass, 'exact': avg_exact})
    humaneval_problem = humaneval_problems[task_id]
    for infilling_problem in task_id_problems:
        prefix, suffix = infilling_problem["prompt_parts"]
        missing_lines = infilling_problem["missing_lines"]
        infill_result = infilling_problem.copy()
        infill_result["text"] = missing_lines
        infill_result["text_untruncated"] = missing_lines
        complete = "\n".join([prefix, missing_lines, suffix])
        infill_result["complete"] = complete
        infill_results.append(infill_result)
        functional_results.append(eval_result(
            task_id, humaneval_problem, truncation_heuristics, infill_result
        ))

100%|█████████████████████████████████████████████████████████████████████████████| 164/164 [19:22<00:00,  7.09s/it, pass=1, exact=1]


In [26]:
from models import TruncationParameters

In [67]:
truncation_parameters = [TruncationParameters.from_heuristics(infill_args.truncation_heuristics, problem["missing_lines"], suffix)]

In [69]:
kwargs = dict(
    verbose=False, n=infill_args.num_candidates,
    bidirectional_generation=infill_args.bidirectional_generation,
    bidirectional_scoring=infill_args.bidirectional_scoring,
    truncation_parameters=truncation_parameters,
    scoring=infill_args.candidate_scoring,
)
# if args.temperature == 0.0:
#     # kwargs.update(sampling=False)
# else:
kwargs.update(sampling=True, top_p=infill_args.top_p, temperature=infill_args.temperature)
# kwargs.update(sampling=True, top_p=infill_args.top_p, temperature=0.8)
# kwargs.update(max_tokens=30)

In [70]:
kwargs

{'verbose': False,
 'n': 1,
 'bidirectional_generation': True,
 'bidirectional_scoring': False,
 'truncation_parameters': [TruncationParameters(max_num_lines=1, suffix='        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False')],
 'scoring': 'random',
 'sampling': True,
 'top_p': 0.95,
 'temperature': 0.2}

In [71]:
model.batch_size = 5

In [72]:
sorted_choices, response = model.rank_infills(prompt_parts, **kwargs)

In [73]:
response

{'prompt_parts': ['from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
  '        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False'],
 'choices': [{'complete': ['from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
    ' 

In [74]:
sorted_choices[0].keys()

dict_keys(['complete', 'infills', 'infills_untruncated', 'logprobs'])

In [75]:
sorted_choices[0]

{'complete': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    numbers = sorted(numbers)\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False',
 'infills': ['    numbers = sorted(numbers)'],
 'infills_untruncated': ['    numbers = sorted(numbers)\n    for idx, elem in enumerate(numbers):\n        if idx != 0:\n            distance = abs(elem - numbers[idx - 1])\n            if distance < threshold:\n                return True\n\n    return False\n\n\ndef has_close_elements_list(numbers: List[float], threshold: floa

In [76]:
def truncate_overlap(infill, suffix, minimum_num_characters=None, minimum_num_suffix_lines=1):
    if minimum_num_characters is None:
        non_empty_suffix_lines = [l.strip() for l in suffix.strip("\n") if l.strip()]
        minimum_num_characters = sum(len(l) for l in non_empty_suffix_lines[:minimum_num_suffix_lines])
    for i in range(len(infill), minimum_num_characters, -1):
        if infill[-i:] == suffix[:i]:
            return infill[:-i]
    return infill

In [59]:
pdb on

Automatic pdb calling has been turned ON


In [63]:
model.score_text([sorted_choices[6]['complete']])

[-174.8389892578125]

In [66]:
model.score_text([choice['complete'] for choice in sorted_choices], 'mean')

[-1.5474575757980347,
 -1.5674219131469727,
 -1.5674219131469727,
 -1.5474575757980347,
 -1.5474575757980347,
 -1.5564860105514526,
 -1.5474575757980347,
 -1.5564860105514526,
 -1.5671440362930298,
 -1.5671440362930298]

In [79]:
choice['complete']

'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    numbers.sort()\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False'

In [65]:
[choice['infills'] for choice in sorted_choices]

[['                if distance < threshold:'],
 ['                if distance > threshold:'],
 ['                if distance > threshold:'],
 ['                if distance < threshold:'],
 ['                if distance < threshold:'],
 ['                if distance <= threshold:'],
 ['                if distance < threshold:'],
 ['                if distance <= threshold:'],
 ['                if distance > threshold:'],
 ['                if distance > threshold:']]

In [30]:
from models import OpenAIModel

In [78]:
codex = OpenAIModel()

[autoreload of models failed: Traceback (most recent call last):
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
TypeError: __class__ assignm

In [17]:
import openai
from secret import API_KEY
openai.api_key = API_KEY

In [18]:
from infill_evaluation import *

In [19]:
infill_problems = list(generate_he_infill_problems(args, "one_line"))

In [20]:
prefix, suffix = infill_problems[0][1][0]['prompt_parts']

In [21]:
print(prefix)

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



In [22]:
print(suffix)

        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


In [23]:
response = openai.Completion.create(
    engine="code-davinci-002",
    prompt=prefix,
    suffix=suffix,
    logprobs=1,
    max_tokens=128,
    temperature=0.2,
    echo=False
)

In [24]:
choice = response['choices'][0]

In [25]:
choice['text']

'    for idx, elem in enumerate(numbers):\n'

In [37]:
from models import make_model

In [49]:
old_model_name = '/checkpoint/dpf/models/cm-6B-armen/checkpoint_last_consolidated.pt'
infill_args.model_name = 'code-davinci-002'

In [50]:
infill_args.prompt_prefix = None

In [ ]:
infill_args.engine='code_'

In [51]:
infill_args

Namespace(batch_size=10, beam=1, bidirectional_generation=True, bidirectional_scoring=False, candidate_scoring='random', eval_type='one_line', evaluate_only=False, git_status=False, max_infill_attempts=1, max_tokens=450, model_name='code-davinci-002', num_candidates=1, prompt_prefix=None, result_base_path=None, temperature=0.2, tokenizer_name='gpt2_pretokenization_newlines_only', top_p=0.95, truncation_heuristics=['num_lines', 'suffix'])

In [62]:
autoreload 2

In [66]:
codex_model = make_model(infill_args)

[autoreload of models failed: Traceback (most recent call last):
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
TypeError: __class__ assignm

In [70]:
response = codex_model.infill([prefix, suffix], temperature=1.0, n=5)

In [71]:
for choice in response['choices']:
    print(choice['text'])

    for idx, elem in enumerate(numbers):

    for idx, elem in enumerate(numbers):

    for idx, elem in enumerate(numbers):

    for idx, elem in enumerate(numbers):

    for idx, elem in enumerate(numbers):



In [57]:
response['choices'][0]['text']

'    for idx, elem in enumerate(numbers):\n'

In [59]:
response['choices'][0]['finish_reason']

'stop'